## Import

In [46]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore') 

## Fixed Random-Seed

In [17]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Load Data

In [18]:
path = "/mnt/d/data/jeju"

train_df = pd.read_csv(os.path.join(path, './train.csv'))
test_df = pd.read_csv(os.path.join(path, './test.csv'))

## Data Pre-Processing

In [19]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x : int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x : int(x[8:10]))

test_df['year'] = test_df['timestamp'].apply(lambda x : int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x : int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x : int(x[8:10]))

In [20]:
#학습에 사용하지 않을 변수들을 제거합니다
train_x = train_df.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)'])
train_y = train_df['price(원/kg)']

test_x = test_df.drop(columns=['ID', 'timestamp'])

In [21]:
#질적 변수들을 수치화합니다
qual_col = ['item', 'corporation', 'location']
encoders = []

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    encoders.append(le)
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

print('Done.')

Done.


## Split data 

In [22]:
split_year = 2023
split_month = 2

valid_index = (train_x["year"] >= split_year) & (train_x["month"] >= split_month)
train_x_ = train_x[~valid_index]
train_y_ = train_y[~valid_index]
valid_x = train_x[valid_index]
valid_y = train_y[valid_index]

## Fit RNN

In [28]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.9.0+cu111].
device:[cuda:0].


In [32]:
train_x

,item,corporation,location,year,month,day
0,4,0,0,2019,1,1
1,4,0,0,2019,1,2
2,4,0,0,2019,1,3
3,4,0,0,2019,1,4
4,4,0,0,2019,1,5
...,...,...,...,...,...,...
59392,3,5,0,2023,2,27
59393,3,5,0,2023,2,28
59394,3,5,0,2023,3,1
59395,3,5,0,2023,3,2


In [44]:
train_x

,item,corporation,location,year,month,day
0,4,0,0,2019,1,1
1,4,0,0,2019,1,2
2,4,0,0,2019,1,3
3,4,0,0,2019,1,4
4,4,0,0,2019,1,5
...,...,...,...,...,...,...
59392,3,5,0,2023,2,27
59393,3,5,0,2023,2,28
59394,3,5,0,2023,3,1
59395,3,5,0,2023,3,2


In [102]:
class BaseDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return len(self.x_data)

In [103]:
dataset = BaseDataset(np.array(train_x), np.array(train_y).reshape(-1, 1))

In [105]:
BATCH_SIZE = 128
train_iter = torch.utils.data.DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)
# test_iter = torch.utils.data.DataLoader(mnist_test,batch_size=BATCH_SIZE,shuffle=True,num_workers=1)

In [108]:
class RecurrentNeuralNetworkClass(nn.Module):
    def __init__(self,name='rnn',xdim=6,hdim=128,ydim=1,n_layer=3):
        super(RecurrentNeuralNetworkClass,self).__init__()
        self.name = name
        self.xdim = xdim
        self.hdim = hdim
        self.ydim = ydim
        self.n_layer = n_layer # K

        self.rnn = nn.LSTM(
            input_size=self.xdim,hidden_size=self.hdim,num_layers=self.n_layer,batch_first=True)
            #num_layers: lstm을 위로 몇번 스택할지
        self.lin = nn.Linear(self.hdim,self.ydim)

    def forward(self,x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.n_layer, x.size(0), self.hdim).to(device)     
                         #s.size(0) = x.shape[0] = 28).to(device)
        c0 = torch.zeros(self.n_layer, x.size(0), self.hdim).to(device)
        # RNN
        rnn_out,(hn,cn) = self.rnn(x, (h0,c0)) 
        # x:[N x L x Q] => rnn_out:[N x L x D]
        # Linear
        out = self.lin(
            rnn_out[:, -1 :]    #가장 마지막(상위)layer 값
        ).view([-1,self.ydim]) 
        return out 

R = RecurrentNeuralNetworkClass(
    name='rnn',xdim=6,hdim=256,ydim=10,n_layer=2).to(device)
loss = nn.CrossEntropyLoss()
optm = optim.Adam(R.parameters(),lr=1e-3)
print ("Done.")

Done.


In [109]:
np.set_printoptions(precision=3)
torch.set_printoptions(precision=3)
x_numpy = np.random.rand(2,1000,6) # [N x L x Q]
x_torch = torch.from_numpy(x_numpy).float().to(device)
rnn_out,(hn,cn) = R.rnn(x_torch) # forward path

print ("rnn_out:",rnn_out.shape) # [N x L x D]
print ("Hidden State hn:",hn.shape) # [K x N x D]
print ("Cell States cn:",cn.shape) # [K x N x D]

rnn_out: torch.Size([2, 1000, 256])
Hidden State hn: torch.Size([2, 2, 256])
Cell States cn: torch.Size([2, 2, 256])


### check params 

In [110]:
n_param = 0
for p_idx,(param_name,param) in enumerate(R.named_parameters()):
    if param.requires_grad:
        param_numpy = param.detach().cpu().numpy() # to numpy array 
        n_param += len(param_numpy.reshape(-1))
        print ("[%d] name:[%s] shape:[%s]."%(p_idx,param_name,param_numpy.shape))
        print ("    val:%s"%(param_numpy.reshape(-1)[:5]))
print ("Total number of parameters:[%s]."%(format(n_param,',d')))

[0] name:[rnn.weight_ih_l0] shape:[(1024, 6)].
    val:[-0.044 -0.04  -0.048 -0.054  0.027]
[1] name:[rnn.weight_hh_l0] shape:[(1024, 256)].
    val:[-0.055  0.038 -0.025  0.031 -0.053]
[2] name:[rnn.bias_ih_l0] shape:[(1024,)].
    val:[ 0.004  0.057 -0.034  0.022  0.022]
[3] name:[rnn.bias_hh_l0] shape:[(1024,)].
    val:[-0.03  -0.006  0.028  0.022 -0.029]
[4] name:[rnn.weight_ih_l1] shape:[(1024, 256)].
    val:[ 0.042 -0.061 -0.006  0.041  0.022]
[5] name:[rnn.weight_hh_l1] shape:[(1024, 256)].
    val:[ 0.022  0.036 -0.061 -0.001 -0.032]
[6] name:[rnn.bias_ih_l1] shape:[(1024,)].
    val:[-0.05   0.004 -0.015  0.061 -0.022]
[7] name:[rnn.bias_hh_l1] shape:[(1024,)].
    val:[ 0.007  0.036  0.023 -0.005 -0.034]
[8] name:[lin.weight] shape:[(10, 256)].
    val:[-0.053  0.059 -0.001 -0.042 -0.052]
[9] name:[lin.bias] shape:[(10,)].
    val:[ 0.015 -0.024  0.019 -0.038  0.01 ]
Total number of parameters:[799,242].


### forward path

In [111]:
x_numpy = np.random.rand(3,59397,6) # [N x L x Q]
x_torch = torch.from_numpy(x_numpy).float().to(device)
y_torch = R.forward(x_torch) # [N x 1 x R] where R is the output dim.
y_numpy = y_torch.detach().cpu().numpy() # torch tensor to numpy array
# print ("x_torch:\n",x_torch)
# print ("y_torch:\n",y_torch)
print ("x_numpy %s"%(x_numpy.shape,))
print ("y_numpy %s"%(y_numpy.shape,))

x_numpy (3, 59397, 6)
y_numpy (3, 10)


### eval

In [112]:
def func_eval(model,data_iter,device):
    with torch.no_grad():
        n_total,n_correct = 0,0
        model.eval() # evaluate (affects DropOut and BN)
        for batch_in,batch_out in data_iter:
            y_trgt = batch_out.to(device)
            model_pred = model.forward(batch_in.view(-1,30,6).to(device))
            _,y_pred = torch.max(model_pred,1)
            n_correct += (y_pred==y_trgt).sum().item()
            n_total += batch_in.size(0)
        val_accr = (n_correct/n_total)
        model.train() # back to train mode 
    return val_accr
print ("Done")

Done


### Train 

In [113]:
print ("Start training.")
R.train() # to train mode 
EPOCHS,print_every = 5,1
for epoch in range(EPOCHS):
    loss_val_sum = 0
    for batch_in,batch_out in train_iter:
        # Forward path
        y_pred = R.forward(batch_in.view(-1,28,28).to(device))
        loss_out = loss(y_pred,batch_out.to(device))
        # Update
        optm.zero_grad() # reset gradient 
        loss_out.backward() # backpropagate
        optm.step() # optimizer update
        loss_val_sum += loss_out
    loss_val_avg = loss_val_sum/len(train_iter)
    # Print
    if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
        train_accr = func_eval(R,train_iter,device)
        test_accr = func_eval(R,test_iter,device)
        print ("epoch:[%d] loss:[%.3f] train_accr:[%.3f] test_accr:[%.3f]."%
               (epoch,loss_val_avg,train_accr,test_accr))
print ("Done")

Start training.


RuntimeError: shape '[-1, 28, 28]' is invalid for input of size 1536